# 4 x 3D EfficientNet Classifiers

In this notebook, we will train **four** 3D convolutional neural networks and combine their predictions in order to recognize a brain tumor.

**3D images**
* each image has the following shape: Channel x Width x Height x Depth (i.e., 1 x Width x Height x Depth)
* *depth:* the depth represents the depth or the number of slices. (I tried various values >= 30. If for a given id, the depth <= the given value (e.g., 30), I replaced the missing depth slices with zero matrices (see [Zabir Al Nazi Nabil](https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train)))
* added some albumentation such as CLAHE, brightness, and CoarseDropout for the training images
* removed black pixels (see [Zabir Al Nazi Nabil](https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train))

**3D EfficientNet** <br>
* Base: [EfficientNet-PyTorch-3D](https://github.com/shijianjian/EfficientNet-PyTorch-3D)
* Custom head: One Fully-Connected Layer

**Slicing Strategy**
* take each x-th slice from each 3D image, whereby x (e.g., 2,4,6) depends on the maximum number of slices

#### Imports

In [ ]:
import os
import random
import glob
import re

import pandas as pd

import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.optim import lr_scheduler

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import cv2

import albumentations as A

from tqdm import tqdm

import wandb

from efficientnet_pytorch_3d import EfficientNet3D

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

#### Seed

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(42)

#### Config

In [ ]:
PATH = '..'

config = dict(
    # Pre-processing
    SLICE_NUMBER = 64, # >= 30
    REMOVE_BLACK_BOUNDARIES = True,
    DICOM=False,
    
    # Albumentation
    RRC_SIZE = 256,
    RRC_MIN_SCALE = 0.85,
    RRC_RATIO = (1., 1.),
    CLAHE_CLIP_LIMIT = 2.0,
    CLAHE_TILE_GRID_SIZE = (8, 8),
    CLAHE_PROB = 0.50,
    BRIGHTNESS_LIMIT = (-0.2,0.2),
    BRIGHTNESS_PROB = 0.40,
    HUE_SHIFT = (-15, 15),
    SAT_SHIFT = (-15, 15),
    VAL_SHIFT = (-15, 15),
    HUE_PROB = 0.64,
    COARSE_MAX_HOLES = 16,
    COARSE_PROB = 0.7,
    
    # Training
    N_EPOCHS = 15,
    BATCH_SIZE = 8,
    LEARNING_RATE = 0.01,
    WEIGHT_DECAY = 0.02,
    OPTIMIZER = "SGD",
    MOMENTUM = 0.9,
    SCHEDULER = "ReduceLROnPlateau",
    
    # Logging
    VERBOSE = False,
    MODELNAME = "04-4x3D-EfficientNet-v2"
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### wandb

In [ ]:
wandb.login()
run = wandb.init(entity="uzk-wim", project='rsna-miccai-4x3D', config=config)
config = wandb.config
wandb.run.name = f"{config.MODELNAME}-{str(config.SLICE_NUMBER)}-{str(config.RRC_SIZE)}"

### 1. Load Data

To create a 3D image, we will pick SLICE_NUMBER middle slices from each of the four MRI types (i.e., FLAIR, T1w, T1wCE, T2w). For example, if we set *SLICE_NUMBER=30*, each 3D image will have the shape: 1 x Width x Height x 30. Further, one image contains only the images from *one* MRI type as opposed to other notebooks where all MRI types are combined in a single image.

* If for a given MRI type, the number of images < SLICE_Number, than we will "fill up" the remaining *number_of_images - SLICE_NUMBER* slices with all black images
* We removed black pixels (for more information, see [here](https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train))


### 1.1 Utilities

#### 1.1.1 Augmentation

In [ ]:
train_transform = A.Compose([
    A.RandomResizedCrop(
        config.RRC_SIZE, config.RRC_SIZE,            
        scale=(config.RRC_MIN_SCALE, 1.0),
        ratio=config.RRC_RATIO,
        p=1.0
    ),
    A.CLAHE(
        clip_limit=config.CLAHE_CLIP_LIMIT,
        tile_grid_size=config.CLAHE_TILE_GRID_SIZE,
        p=config.CLAHE_PROB
    ),
    A.RandomBrightnessContrast(
        brightness_limit=config.BRIGHTNESS_LIMIT,
        p=config.BRIGHTNESS_PROB
    ),
    A.HueSaturationValue(
        hue_shift_limit=config.HUE_SHIFT, 
        sat_shift_limit=config.SAT_SHIFT, 
        val_shift_limit=config.VAL_SHIFT, 
        p=config.HUE_PROB
    ),
    A.CoarseDropout(
        max_holes=config.COARSE_MAX_HOLES,
        p=config.COARSE_PROB),
])

valid_transform = A.Compose([
    A.RandomResizedCrop( 
        config.RRC_SIZE, config.RRC_SIZE,            
        scale=(config.RRC_MIN_SCALE, 1.0),
        ratio=config.RRC_RATIO,
        p=1.0
    )
])

#### 1.1.2 Loading Images

In [ ]:
def dicom_2_image(file, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(file)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        img = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        img = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    return img

def remove_black_boundaries(img):
    (x, y) = np.where(img > 0)
    if len(x) > 0 and len(y) > 0:
        x_mn = np.min(x)
        x_mx = np.max(x)
        y_mn = np.min(y)
        y_mx = np.max(y)
        if (x_mx - x_mn) > 10 and (y_mx - y_mn) > 10:
            img = img[:,np.min(y):np.max(y)]
    return img

def get_idxs(img_depth):
    minimum_idx = 0
    maximum_idx = img_depth
    step = 1
    if config.SLICE_NUMBER < img_depth:
        mod = img_depth % config.SLICE_NUMBER
        threshold = int(mod // 2)
        minimum_idx = threshold
        maximum_idx = img_depth - threshold
        step = img_depth // config.SLICE_NUMBER
    if config.VERBOSE:
        print(f"Minimum {minimum_idx}")
        print(f"Maximum {maximum_idx}")
        print(f"Step size: {step}")
    return minimum_idx, maximum_idx, step

def get_3d_image(mri_type, aug, dicom):
    img_depth = len(mri_type)
    if config.VERBOSE:
        print(f"Length of folder: {img_depth}")
    minimum_idx, maximum_idx, step = get_idxs(img_depth)
    # Create array which contains the images
    mri_img = []
    counter = 0
    for i in range(minimum_idx, maximum_idx, step):
        file = mri_type[i]
        if dicom:
            img = dicom_2_image(file)
        else:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        # Remove black boundaries
        if config.REMOVE_BLACK_BOUNDARIES:
            img = remove_black_boundaries(img)
        if aug:
            transformed = train_transform(image=img)
            img = transformed["image"]
        else:
            transformed = valid_transform(image=img)
            img = transformed["image"]
        mri_img.append(np.array(img))
        counter += 1
        if counter == config.SLICE_NUMBER:
            break
    mri_img = np.array(mri_img).T
    # If less than SLICE_NUMBER slices, add SLICE_NUMBER - mri_img.shape[-1] images with only zero values
    if mri_img.shape[-1] < config.SLICE_NUMBER:
        if config.VERBOSE:
            print(f"Current slices: {mri_img.shape[-1]}")
        n_zero = config.SLICE_NUMBER - mri_img.shape[-1]
        mri_img = np.concatenate((mri_img, np.zeros((config.RRC_SIZE, config.RRC_SIZE, n_zero))), axis = -1)
    if config.VERBOSE:
        print(f"Shape of mri_img: {mri_img.shape}")
    return mri_img

def load_images(scan_id, mri_type, aug=True, split="train", dicom=False):
    file_ext = "png"
    if dicom:
        file_ext = "dcm"
    if config.VERBOSE:
        print(f"Scan id {scan_id}")
        
    # Ascending sort
    if mri_type == "FLAIR":
        flair = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/FLAIR/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
        img = get_3d_image(flair, aug, dicom)
    elif mri_type == "T1w":
        t1w = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T1w/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
        img = get_3d_image(t1w, aug, dicom)
    elif mri_type == "T1wCE":
        t1wce = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T1wCE/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
        img = get_3d_image(t1wce, aug, dicom)
    else:
        t2w = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T2w/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
        img = get_3d_image(t2w, aug, dicom)
    
    # Return 3D image: ChannelsxWidthxHeightxDepth
    img = img.reshape((1,config.RRC_SIZE,config.RRC_SIZE,config.SLICE_NUMBER))
    
    # Normalize
    img = img - img.min()
    img = (img + 1e-5) / (img.max() - img.min() + 1e-5)
    
    return img

#### 1.1.3 Example

In [ ]:
scan_id = "00000"
slice_nb = 10

figure, axes = plt.subplots(2,4, figsize=(15,15))

flair_3d_image = load_images(scan_id, "FLAIR", aug=True)
flair_3d_image_fs = flair_3d_image[0,:,:,slice_nb]
t1w_3d_image = load_images(scan_id, "T1w", aug=True)
t1w_3d_image_fs = t1w_3d_image[0,:,:,slice_nb]
t1wce_3d_image = load_images(scan_id, "T1wCE", aug=True)
t1wce_3d_image_fs = t1wce_3d_image[0,:,:,slice_nb]
t2w_3d_image = load_images(scan_id, "T2w", aug=True)
t2w_3d_image_fs = t2w_3d_image[0,:,:,slice_nb]

im = axes[0,0].imshow(flair_3d_image_fs)
im = axes[0,1].imshow(t1w_3d_image_fs)
im = axes[0,2].imshow(t1wce_3d_image_fs)
im = axes[0,3].imshow(t2w_3d_image_fs)

for ax in axes[0]:
    ax.set_xticks([])
    ax.set_yticks([])

flair_3d_image = load_images(scan_id, "FLAIR", aug=False)
flair_3d_image_fs = flair_3d_image[0,:,:,slice_nb]
t1w_3d_image = load_images(scan_id, "T1w", aug=False)
t1w_3d_image_fs = t1w_3d_image[0,:,:,slice_nb]
t1wce_3d_image = load_images(scan_id, "T1wCE", aug=False)
t1wce_3d_image_fs = t1wce_3d_image[0,:,:,slice_nb]
t2w_3d_image = load_images(scan_id, "T2w", aug=False)
t2w_3d_image_fs = t2w_3d_image[0,:,:,slice_nb]

im = axes[1,0].imshow(flair_3d_image_fs)
im = axes[1,1].imshow(t1w_3d_image_fs)
im = axes[1,2].imshow(t1wce_3d_image_fs)
im = axes[1,3].imshow(t2w_3d_image_fs)

for ax in axes[1]:
    ax.set_xticks([])
    ax.set_yticks([])

figure.tight_layout()
plt.show()

### 1.2 Dataset and Dataloader

Create a PyTorch Dataset and DataLoader **for each** MRI type

In [ ]:
# Load ids and labels and make a stratified 80:20 split
df = pd.read_csv("../train_labels.csv")
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df["MGMT_value"])

In [ ]:
class RSNADataset(Dataset):
    def __init__(self, ids, labels, mri_type="FLAIR", split="train", dicom=False):
        self.ids = ids
        self.labels = labels
        self.mri_type = mri_type
        self.split = split
        self.dicom = dicom
        
        if split == "train":
            remove_ids = [709, 109, 123]
            self.ids = [id_ for id_ in self.ids if id_ not in remove_ids]  
    
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        patient_id = self.ids[idx]
        patient_id = str(patient_id).zfill(5)
        if self.split == "train":
            imgs = load_images(patient_id, self.mri_type, aug=True, dicom=self.dicom)
        elif self.split == "valid":
            imgs = load_images(patient_id, self.mri_type, aug=False, dicom=self.dicom)
        else:
            imgs = load_images(patient_id, self.mri_type, aug=False, split=self.split, dicom=self.dicom)

        if self.split != "test":
            return torch.tensor(imgs, dtype = torch.float32), torch.tensor(self.labels[idx], dtype = torch.long)
        else:
            return torch.tensor(imgs, dtype = torch.float32), torch.tensor(self.ids[idx], dtype = torch.long)

In [ ]:
def get_dataloader(df, mri_type, shuffle=True, split="train"):
    ds = RSNADataset(df["BraTS21ID"].to_numpy(), df["MGMT_value"].to_numpy(), mri_type=mri_type, split=split, dicom=config.DICOM)
    dl = DataLoader(ds, batch_size=config.BATCH_SIZE, shuffle=shuffle)
    return dl

In [ ]:
# FLAIR DataLoader
train_flair_dl = get_dataloader(df_train, "FLAIR")
valid_flair_dl = get_dataloader(df_valid, "FLAIR", False, "valid")

# T1w DataLoader
train_t1w_dl = get_dataloader(df_train, "T1w")
valid_t1w_dl = get_dataloader(df_valid, "T1w", False, "valid")

# T1wCE DataLoader
train_t1wce_dl = get_dataloader(df_train, "T1wCE")
valid_t1wce_dl = get_dataloader(df_valid, "T1wCE", False, "valid")

# T2w DataLoader
train_t2w_dl = get_dataloader(df_train, "T2w")
valid_t2w_dl = get_dataloader(df_valid, "T2w", False, "valid")

In [ ]:
images, labels = next(iter(train_flair_dl))
print(f"Shape of the batch {images.shape}")
print(f"Batch size: {images.shape[0]}")
print(f"Number of channels each image has: {images.shape[1]}")
print(f"Size of each image is: {images.shape[2]}x{images.shape[3]}")
print(f"Depth of each channel/sequence: {images.shape[-1]}")

### 1.3 Model

#### Custom GliobCNN with 3D EfficientNet as base

* Base: [EfficientNet-PyTorch-3D](https://github.com/shijianjian/EfficientNet-PyTorch-3D)
* Custom head: One Fully-Connected Layer

In [ ]:
class GliobCNN(nn.Module):
    def __init__(self):
        super(GliobCNN, self).__init__()
        
        self.efficient = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        self.efficient._fc = nn.Linear(in_features=self.efficient._fc.in_features, out_features=2, bias=True)
        
    def forward(self, X):
        out = self.efficient(X)
        return out

### 1.4 Training

In [ ]:
def training(net, n_epochs, optimizer, loss_function, scheduler, train_dl, valid_dl, mri_type):
    wandb.watch(net, loss_function, log='all', log_freq=10)
    
    best_loss = np.inf

    # Iterate over the dataset n_epochs times
    for epoch in range(n_epochs):
        net.train()  # net.train() will notify all your layers that you are in training mode

        train_loss = 0  # Training loss in epoch
        y_train_list = []
        y_hat_train_list = []
        roc_train = 0.0

        # For each batch, pass the training examples, calculate loss and gradients and optimize the parameters
        for xb, yb in tqdm(train_dl, desc="Training"):
            optimizer.zero_grad()  # zero_grad clears old gradients from the last step

            xb = xb.to(device)
            yb = yb.to(device)

            y_hat = net(xb)  # Forward pass
            loss = loss_function(y_hat, yb)  # Calculate Loss

            loss.backward()  # Calculate the gradients (using backpropagation)
            optimizer.step()  # # Optimize the parameters: opt.step() causes the optimizer to take a step based on the gradients of the parameters.

            train_loss += loss.item()
            y_train_list.extend(yb.tolist())
            y_hat = F.softmax(y_hat)
            y_hat_train_list.extend(y_hat[:,1].tolist())
            
            if config.SCHEDULER == "OneCycleLR":
                scheduler.step()
                
        y_hat_train_list = [1 if x > 0.5 else 0 for x in y_hat_train_list]
        roc_train = roc_auc_score(y_train_list, y_hat_train_list)
        
        train_loss = train_loss/len(train_dl)
                
        wandb.log({"epoch": epoch, "train_loss": train_loss, "train_roc": roc_train})

        valid_loss = 0  # Validation loss in epoch
        y_valid_list = []
        y_hat_valid_list = []
        roc_valid = 0.0

        net.eval()  # net.eval() will notify all your layers that you are in evaluation mode
        with torch.no_grad():
            # Perform a prediction on the validation set  
            for xb_valid, yb_valid in tqdm(valid_dl, desc="Validation"):
                xb_valid = xb_valid.to(device)
                yb_valid = yb_valid.to(device)

                y_hat = net(xb_valid)  # Forward pass
                loss = loss_function(y_hat, yb_valid)  # Calculate Loss

                valid_loss += loss.item()
                y_valid_list.extend(yb_valid.tolist())
                y_hat = F.softmax(y_hat)
                y_hat_valid_list.extend(y_hat[:,1].tolist())

        y_hat_valid_list = [1 if x > 0.5 else 0 for x in y_hat_valid_list]
        roc_valid = roc_auc_score(y_valid_list, y_hat_valid_list)
        
        valid_loss = valid_loss/len(valid_dl)
        
        if config.SCHEDULER == "ReduceLROnPlateau":
            scheduler.step(valid_loss)

        wandb.log({"epoch": epoch, "valid_loss": valid_loss, "valid_roc": roc_valid})
        
        if best_loss > valid_loss:
            best_loss = valid_loss
            wandb.run.summary["best_loss"] = best_loss
            wandb.run.summary["best_roc"] = roc_valid
            torch.save(net.state_dict(), f'../models/{config.MODELNAME}-{mri_type}-loss-{round(valid_loss, 3)}.pt')

In [ ]:
# Create Loss Function
loss_function = nn.CrossEntropyLoss()

def utilities(steps_per_epoch):
    # Create model
    model = GliobCNN()

    # Create optimizer
    if config.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=config.LEARNING_RATE, momentum=config.MOMENTUM)
    elif config.OPTIMIZER == "Adam":
        optimizer = torch.optim.Adam(model.parameters(),lr = config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
    else:
        optimizer = torch.optim.AdamW(model.parameters(),lr = config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)

    # Scheduler
    #  The 1cycle policy by fasti: https://sgugger.github.io/the-1cycle-policy.html
    if config.SCHEDULER == "OneCycleLR":
        scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.03, steps_per_epoch=steps_per_epoch, epochs=config.N_EPOCHS)
    else:
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.25)

    model.to(device)
    
    return model, optimizer, scheduler

#### 1.4.1 FLAIR model

In [ ]:
wandb.run.name = f"{config.MODELNAME}-FLAIR"

flair_model, optimizer, scheduler = utilities(len(train_flair_dl))

training(flair_model, config.N_EPOCHS, optimizer, loss_function, scheduler, train_flair_dl, valid_flair_dl, mri_type="FLAIR")

run.finish()

#### 1.4.2 T1w model

In [ ]:
run = wandb.init(entity="uzk-wim", project='rsna-miccai-4x3D', config=config)
wandb.run.name = f"{config.MODELNAME}-T1w"

t1w_model, optimizer, scheduler = utilities(len(train_t1w_dl))

training(t1w_model, config.N_EPOCHS, optimizer, loss_function, scheduler, train_t1w_dl, valid_t1w_dl, mri_type="T1w")

run.finish()

#### 1.4.3 T1wCE model

In [ ]:
run = wandb.init(entity="uzk-wim", project='rsna-miccai-4x3D', config=config)
wandb.run.name = f"{config.MODELNAME}-T1wCE"

t1wce_model, optimizer, scheduler = utilities(len(train_t1wce_dl))

training(t1wce_model, config.N_EPOCHS, optimizer, loss_function, scheduler, train_t1wce_dl, valid_t1wce_dl, mri_type="T1wCE")

run.finish()

#### 1.4.4 T2w model

In [ ]:
run = wandb.init(entity="uzk-wim", project='rsna-miccai-4x3D', config=config)
wandb.run.name = f"{config.MODELNAME}-T2w"

t2w_model, optimizer, scheduler = utilities(len(train_t2w_dl))

training(t2w_model, config.N_EPOCHS, optimizer, loss_function, scheduler, train_t2w_dl, valid_t2w_dl, mri_type="T2w")

run.finish()

### 1.5 Evaluation

In [ ]:
def evaluation(net, valid_dl):
    
    ys_hats = None
    
    net.eval()  # net.eval() will notify all your layers that you are in evaluation mode
    with torch.no_grad():
        # Perform a prediction on the validation set  
        for xb_valid, yb_valid in valid_dl:
            xb_valid = xb_valid.to(device)
            yb_valid = yb_valid.to(device)

            y_hat = net(xb_valid)  # Forward pass
            y_hat = F.softmax(y_hat)
            if ys_hats is None:
                ys_hats = y_hat.cpu().detach().numpy()
            else:
                ys_hats = np.concatenate((ys_hats, y_hat.cpu().detach().numpy()), axis=0)

    return ys_hats

In [ ]:
flair_y_hat = evaluation(flair_model, valid_flair_dl)
t1w_y_hat = evaluation(t1w_model, valid_t1w_dl)
t1wce_y_hat = evaluation(t1wce_model, valid_t1wce_dl)
t2w_y_hat = evaluation(t2w_model, valid_t2w_dl)

In [ ]:
df = df_valid.copy()
#df = df.loc[(df.BraTS21ID != 123) & (df.BraTS21ID != 109)]
df["FLAIR"] = flair_y_hat[:,1].tolist()
df["T1w"] = t1w_y_hat[:,1].tolist()
df["T1wCE"] = t1wce_y_hat[:,1].tolist()
df["T2w"] = t2w_y_hat[:,1].tolist()
df["y_hat"] = (df["FLAIR"] + df["T1w"] + df["T1wCE"] + df["T2w"]) / 4
print(roc_auc_score(df["MGMT_value"], df["y_hat"]))

Done